In [ ]:
from acr.utils import SOM_BLUE, ACR_BLUE, NNXR_GRAY

MAIN_EXP = 'swi'
SUBJECT_TYPE = 'som'
MAIN_COLOR = SOM_BLUE

In [ ]:
#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import kdephys as kde
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import acr
import xarray as xr
from scipy import stats
import pingouin as pg
from scipy.stats import shapiro, normaltest
# ---------------------------- EXTRAS --------------------------------#
from kdephys.plot.main import _title, bp_plot
import kdephys.utils.spectral as sp
bands = sp.bands

import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
from acr.utils import *
pu = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py', 'pu')
dag = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/data_agg.py', 'dag')

In [ ]:
plt.rcdefaults()
acr.plots.supl()

In [ ]:
subjects, exps = pu.get_subject_list(type=SUBJECT_TYPE, exp=MAIN_EXP)
notebook_figure_root = f'{pu.PAPER_FIGURE_ROOT}/OFF_PERIODS'

In [ ]:
full_hyps = {}
hyp_dicts = {}
for subject, exp in zip(subjects, exps):
    full_hyps[subject] = acr.io.load_hypno_full_exp(subject, exp)
    hyp_dicts[subject] = acr.hypnogram_utils.create_acr_hyp_dict(subject, exp, true_stim=True)

In [ ]:
ooscs = {}
for subject, exp in zip(subjects, exps):
    oosc = dag.load_base_oodf(subject, exp, dftype='single-chan')
    ooscs[subject] = acr.onoffmua.true_strictify_oodf(oosc, min_on=0.03)

In [ ]:
# label
for subject in subjects:
    print(subject)
    oodf = ooscs[subject]
    oodf = acr.oo_utils.label_oodf_with_states(oodf, full_hyps[subject])
    oodf = acr.oo_utils.label_oodf_full_bl(oodf)
    oodf = acr.hypnogram_utils.label_df_with_hypno_conditions(oodf, hyp_dicts[subject], col='end_datetime')
    ooscs[subject] = oodf

In [ ]:
oosc_onoff = pl.concat(ooscs.values())
oosc = oosc_onoff.offs()
counts_raw = oosc.group_by(['subject', 'probe', 'channel', 'condition']).agg(pl.col('status').count().alias('count')).sort('subject', 'probe', 'channel', 'condition')

In [ ]:
dfs = []
for subject in subjects:
    durs = []
    hd = hyp_dicts[subject]
    for condition in hd.keys():
        duration = hd[condition]['duration'].sum().total_seconds() if condition != 'stim' else hd[condition]['duration'].sum()
        durs.append(duration)
    dfs.append(pl.DataFrame({'subject': subject, 'condition': list(hd.keys()), 'duration': durs}))
durdf = pl.concat(dfs)

# Relative

In [ ]:
counts = counts_raw.join(durdf, on=['subject', 'condition'], how='left')
counts = counts.with_columns((pl.col('count')/pl.col('duration')).alias('freq'))
scd = {
    'ACR_25': [13],
    'ACR_26': [13], 
    'ACR_29': [8, 9],
    'ACR_30': [11],
    'ACR_40': [9, 10],
    'ACR_41': [13],
    'ACR_44': [8, 9],
}
scd = {}
counts = pu.drop_sub_channels(counts, scd)

In [ ]:
crel = dag.relativize_df(counts, ref_to_col='condition', ref_to_val='late_sd', avg_method='mean', col_to_relativize='freq', on=['subject', 'probe', 'channel'])
crel = acr.info_pipeline.label_df_sub_infra(crel)
cstim = crel.cdn('stim').group_by(['subject', 'probe', 'label']).agg(pl.col('freq_rel').mean())

In [ ]:
nnxo_infra = cstim.filter(pl.col('label')=='infragranular').sort(['subject', 'probe']).prb('NNXo')['freq_rel'].to_numpy()
nnxr_infra = cstim.filter(pl.col('label')=='infragranular').sort(['subject', 'probe']).prb('NNXr')['freq_rel'].to_numpy()
nnxo_sub = cstim.filter(pl.col('label')=='subgranular').sort(['subject', 'probe']).prb('NNXo')['freq_rel'].to_numpy()
nnxr_sub = cstim.filter(pl.col('label')=='subgranular').sort(['subject', 'probe']).prb('NNXr')['freq_rel'].to_numpy()
infra_rel = nnxo_infra/nnxr_infra
sub_rel = nnxo_sub/nnxr_sub
nnxo_gran = cstim.filter(pl.col('label')=='granular').sort(['subject', 'probe']).prb('NNXo')['freq_rel'].to_numpy()
nnxr_gran = cstim.filter(pl.col('label')=='granular').sort(['subject', 'probe']).prb('NNXr')['freq_rel'].to_numpy()

In [ ]:
fig_id = 'off-freq-during-single-chan-SUB_VS_INFRA_optrode-only'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f, ax = acr.plots.gen_paired_boxplot(nnxo_infra, nnxo_sub, colors=[MAIN_COLOR, MAIN_COLOR], alphas=[0.9, 0.9])
ax.set_xticklabels(['Supragranular', 'Infragranular'])
plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
nnxr = nnxo_infra
nnxo = nnxo_sub
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

#stats = pg.ttest(nnxr, nnxo, paired=True)
stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    #acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    acr.stats.write_stats_result(stats_name, 'wilcoxon', stats['W-val'][0], stats['p-val'][0], 'r, RBC', [r, stats['RBC'][0]])
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

# Raw

In [ ]:
crel = dag.relativize_df(counts, ref_to_col='condition', ref_to_val='late_sd', avg_method='mean', col_to_relativize='freq', on=['subject', 'probe', 'channel'])
crel = acr.info_pipeline.label_df_sub_infra(crel)
cstim = crel.cdn('stim').group_by(['subject', 'probe', 'label']).agg(pl.col('freq').mean())

In [ ]:
nnxo_infra = cstim.filter(pl.col('label')=='infragranular').sort(['subject', 'probe']).prb('NNXo')['freq'].to_numpy()
nnxr_infra = cstim.filter(pl.col('label')=='infragranular').sort(['subject', 'probe']).prb('NNXr')['freq'].to_numpy()
nnxo_sub = cstim.filter(pl.col('label')=='subgranular').sort(['subject', 'probe']).prb('NNXo')['freq'].to_numpy()
nnxr_sub = cstim.filter(pl.col('label')=='subgranular').sort(['subject', 'probe']).prb('NNXr')['freq'].to_numpy()
infra_rel = nnxo_infra/nnxr_infra
sub_rel = nnxo_sub/nnxr_sub
nnxo_gran = cstim.filter(pl.col('label')=='granular').sort(['subject', 'probe']).prb('NNXo')['freq'].to_numpy()
nnxr_gran = cstim.filter(pl.col('label')=='granular').sort(['subject', 'probe']).prb('NNXr')['freq'].to_numpy()

In [ ]:
fig_id = 'off-freq-during-single-chan-SUB_VS_INFRA_raw-frequency'
fig_name = f'{SUBJECT_TYPE}__{MAIN_EXP}__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'

f, ax = acr.plots.gen_paired_boxplot(nnxo_infra, nnxo_sub, colors=[MAIN_COLOR, MAIN_COLOR], alphas=[0.9, 0.9])
ax.set_xticklabels(['Supragranular', 'Infragranular'])

plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)


In [ ]:
# =============================
# ========== STATS ============
# =============================
nnxr = nnxo_infra
nnxo = nnxo_sub
write = True

diffs = nnxr - nnxo
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

stats = pg.ttest(nnxr, nnxo, paired=True)
#stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

#r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))


if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'paired_ttest', stats['T'][0], stats['p-val'][0], 'g', hg)
    #acr.stats.write_stats_result(stats_name, 'wilcoxon', stats['W-val'][0], stats['p-val'][0], 'r, RBC', [r, stats['RBC'][0]])
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

# ALL CHANS

In [ ]:
for subject in subjects:
    f, ax = plt.subplots(1, 1, figsize=(16, 6))
    sns.barplot(data=crel.filter(pl.col('subject')==subject).cdn('stim'), x='channel', y='freq_rel', hue='probe', hue_order=['NNXr', 'NNXo'], palette=[NNXR_GRAY, MAIN_COLOR], ax=ax)
    ax.set_title(f'{subject} OFF Frequency')
    plt.show()